## Imports

In [2]:
from collections import OrderedDict
import configparser
from functools import partial
import numpy as np
from shapely import Polygon
import math
from typing import Any
from matplotlib.backend_bases import MouseEvent, MouseButton
from matplotlib.gridspec import GridSpec
from matplotlib import pyplot as plt

import os
import sys
sys.path.insert(1, os.path.realpath(os.path.pardir))

from bbtoolkit.preprocessing.environment.viz import plot_arrow, plot_polygon
from bbtoolkit.data import Cached
from bbtoolkit.data.configparser import EvalConfigParser
from bbtoolkit.preprocessing.environment import Environment
from bbtoolkit.preprocessing.environment.compilers import DynamicEnvironmentCompiler
from bbtoolkit.preprocessing.environment.compilers.callbacks import TransparentObjects
from bbtoolkit.preprocessing.environment.utils import env2builder
from bbtoolkit.preprocessing.environment.visible_planes import LazyVisiblePlaneWithTransparancy
from bbtoolkit.preprocessing.neural_generators import TCGenerator
from bbtoolkit.structures.geometry import Texture, TexturedPolygon
from bbtoolkit.dynamics.callbacks import BaseCallback
from bbtoolkit.preprocessing.environment.fov import FOVManager
from bbtoolkit.preprocessing.environment.fov.ego import EgoManager
from bbtoolkit.math import pol2cart
from bbtoolkit.math.geometry import calculate_polar_distance
from bbtoolkit.dynamics import DynamicsManager
from bbtoolkit.dynamics.callbacks.fov import EgoCallback, EgoSegmentationCallback, FOVCallback, ParietalWindowCallback
from bbtoolkit.dynamics.callbacks.movement import MovementCallback, MovementSchedulerCallback, TrajectoryCallback
from bbtoolkit.movement import MovementManager
from bbtoolkit.movement.trajectory import AStarTrajectory


In [21]:
from abc import ABC, abstractmethod


class AbstractAttention(ABC):
    """
    An abstract class that defines the basic structure for implementing attention mechanisms.

    Methods:
        __call__(objects: list[np.ndarray], return_index: bool = False) -> np.ndarray:
            Abstract method that must be implemented by subclasses. It defines how the attention mechanism
            operates on a list of objects.
    """
    @abstractmethod
    def __call__(self, objects: list[np.ndarray], return_index: bool = False) -> np.ndarray:
        pass


class RhythmicAttention(AbstractAttention):
    """
    An implementation of the AbstractAttention class that selects objects to pay attention to based on a rhythmic pattern.

    Attributes:
        freq (float): The frequency of the attention cycle.
        dt (float): The time step between updates.
        n_objects (int): The number of objects to consider for attention.
        cycle (int): The number of time steps in one complete attention cycle.
        timer (int): A counter used to determine the current position within the attention cycle.
        attend_to (int or None): The index of the currently attended object. None if no object is being attended.
        attention_priority (np.ndarray): An array indicating the priority of each object for receiving attention.

    Methods:
        step() -> bool:
            Advances the timer and determines if the attention cycle is complete.
        visible_objects(objects: list[np.ndarray]) -> list[int]:
            Determines which objects are visible (i.e., have a non-zero size).
        __call__(objects: list[np.ndarray], return_index: bool = False) -> np.ndarray:
            Processes a list of objects and determines which object to pay attention to based on the rhythmic pattern.
    """
    def __init__(self, freq: float, dt: float, n_objects: int):
        """
        Initializes the RhythmicAttention object with the specified frequency, time step, and number of objects.

        Args:
            freq (float): The frequency of the attention cycle.
            dt (float): The time step between updates.
            n_objects (int): The number of objects to consider for attention.
        """
        self.cycle = int(1/(freq * dt))
        self.timer = 0
        self.n_objects = n_objects
        self.attend_to = None
        self.attention_priority = np.zeros(n_objects)

    def step(self) -> bool:
        """
        Advances the timer and determines if the attention cycle is complete.

        Returns:
            bool: True if the cycle is complete, False otherwise.
        """
        self.timer += 1
        self.timer %= self.cycle
        return self.timer == 0

    @staticmethod
    def visible_objects(objects: list[np.ndarray]) -> list[int]:
        """
        Determines which objects are visible (i.e., have a non-zero size).

        Args:
            objects (list[np.ndarray]): A list of objects represented as numpy arrays.

        Returns:
            list[int]: A list indicating the visibility of each object (True for visible, False for not visible).
        """
        return np.array([arr.size > 0 for arr in objects])

    def __call__(self, objects: list[np.ndarray], return_index: bool = False) -> np.ndarray:
        """
        Processes a list of objects and determines which object to pay attention to based on the rhythmic pattern.

        Args:
            objects (list[np.ndarray]): A list of objects represented as numpy arrays.
            return_index (bool, optional): If True, returns the index of the attended object instead of the object itself.
                                           Defaults to False.

        Returns:
            np.ndarray: The attended object or its index, depending on the value of `return_index`.
        """
        # True for all objects that are in the field of view in the current moment
        visible_objects = self.visible_objects(objects)
        # If there are no objects to pay attention to, pay attention to the first visible object or do not pay attention to any object
        if self.attend_to is None:
            visible_objects_indices = np.where(visible_objects)[0]

            if visible_objects_indices.size:
                self.attend_to = visible_objects_indices[0]

        # Increase the priority of the objects that are visible, zero the priority of the attended object and invisible objects
        if self.attend_to is not None:
            self.attention_priority = self.attention_priority*visible_objects + visible_objects
            self.attention_priority[self.attend_to] = 0

        if self.step(): # If the cycle is complete, attend to the other object with the highest priority or attend to no object
            self.attend_to = np.argmax(self.attention_priority) if not np.all(np.logical_not(self.attention_priority)) else None

        # Return the object that is currently being attended to
        if return_index:
            return self.attend_to
        else:
            return objects[self.attend_to] if self.attend_to is not None else np.array([])

In [23]:
from typing import Mapping


class AttentionCallback(BaseCallback):
    """
    A callback class that integrates an attention mechanism into a simulation or model by updating the agent's
    attention at the beginning of each simulation step.

    Attributes:
        attn_manager (AbstractAttention): An instance of an AbstractAttention or its subclass that manages the
                                          attention mechanism.

    Methods:
        set_cache(cache: Mapping):
            Sets the cache for the callback and initializes required keys for the field of view.
        on_step_begin(step: int):
            Updates the agent's attention at the beginning of each simulation step based on the current position
            and direction.
    """
    def __init__(self, attn_manager: AbstractAttention):
        """
        Initializes the AttentionCallback with the specified attention manager.

        Args:
            attn_manager (AbstractAttention): An instance of an AbstractAttention or its subclass that manages the
                                              attention mechanism.
        """

        super().__init__()
        self.attn_manager = attn_manager

    def set_cache(self, cache: Mapping):
        """
        Sets the cache for the callback and initializes required keys for the field of view.

        Args:
            cache (Mapping): A mapping object to be used as the cache for the callback.
        """
        super().set_cache(cache)
        self.cache['attend_to'] = None
        self.cache['attention_priority'] = None
        self.requires = ['objects_fov', 'attend_to', 'attention_priority']

    def on_step_begin(self, step: int):
        """
        Updates the agent's attention at the beginning of each simulation step based on the current position and direction.

        Args:
            step (int): The current step of the simulation.
        """
        self.cache['attend_to'] = self.attn_manager(self.cache['objects_fov'], return_index=True)
        self.cache['attention_priority'] = self.attn_manager.attention_priority

### Define Transformation Circuit and Environment

In [24]:
hd_config_path = '../cfg/cells/hd_cells.ini'
hd_config = EvalConfigParser(interpolation=configparser.ExtendedInterpolation(), allow_no_value=True)
hd_config.read(hd_config_path)

mtl_config_path = '../cfg/cells/mtl_cells.ini'
mtl_config = EvalConfigParser(interpolation=configparser.ExtendedInterpolation(), allow_no_value=True)
mtl_config.read(mtl_config_path)

tr_config_path = '../cfg/cells/transformation_circuit.ini'
tr_config = EvalConfigParser(interpolation=configparser.ExtendedInterpolation(), allow_no_value=True)
tr_config.read(tr_config_path)

env_cfg = EvalConfigParser(interpolation=configparser.ExtendedInterpolation(), allow_no_value=True)
env_cfg.read('../cfg/envs/squared_room.ini')

space_cfg = mtl_config['Space']
h_res = space_cfg.eval('res')
r_max = space_cfg.eval('r_max')

mtl_grid_cfg = mtl_config['PolarGrid']
n_radial_points = mtl_grid_cfg.eval('n_radial_points')
polar_dist_res = mtl_grid_cfg.eval('polar_dist_res')
polar_ang_res = mtl_grid_cfg.eval('polar_ang_res', locals={'pi': np.pi})
h_sig = mtl_grid_cfg.eval('sigma_hill')

tr_space_cfg = tr_config['Space']
tr_res = tr_space_cfg.eval('tr_res', locals={'pi': np.pi})
res = tr_space_cfg.eval('res')

n_steps = tr_config['Training'].eval('n_steps')

hd_neurons_cfg = hd_config['Neurons']
sigma_angular = hd_neurons_cfg.eval('sigma', locals={'pi': np.pi})
n_hd = hd_neurons_cfg.eval('n_neurons')


training_rect_cfg = env_cfg['TrainingRectangle']
max_train_x = training_rect_cfg.eval('max_train_x')
min_train_x = training_rect_cfg.eval('min_train_x')
max_train_y = training_rect_cfg.eval('max_train_y')
min_train_y = training_rect_cfg.eval('min_train_y')

env = Environment.load('../data/envs/main_environment.pkl')

tc_gen = TCGenerator(
    n_hd,
    tr_res,
    res,
    r_max,
    polar_dist_res,
    n_radial_points,
    polar_ang_res,
    h_sig,
    sigma_angular,
    n_steps
)

builder = env2builder(env)
cache_manager = Cached(cache_storage=OrderedDict(), max_size=10000)
compiler = DynamicEnvironmentCompiler(
    builder,
    partial(
        LazyVisiblePlaneWithTransparancy,
        cache_manager=cache_manager,
    ),
    callbacks=TransparentObjects()
)

compiler.add_object(
    TexturedPolygon(
        Polygon([
            (-5, -5),
            (-6, -5),
            (-6, -6),
            (-5, -6)
        ]),
        texture=Texture(
            id_=3,
            color='#ffd200',
            name='main_object'
        )
    ),
    TexturedPolygon(
        Polygon([
            (-7, -7),
            (-8, -7),
            (-8, -8),
            (-7, -8)
        ]),
        texture=Texture(
            id_=3,
            color='#ffd200',
            name='main_object'
        )
    ),
    TexturedPolygon(
        Polygon([
            (2, 2),
            (1, 2),
            (1, 1),
            (2, 1)
        ]),
        texture=Texture(
            id_=3,
            color='#ffd200',
            name='main_object'
        )
    ),
    TexturedPolygon(
        Polygon([
            (-2, 2),
            (-1, 2),
            (-1, 1),
            (-2, 1)
        ]),
        texture=Texture(
            id_=3,
            color='#ffd200',
            name='main_object'
        )
    ),
    TexturedPolygon(
        Polygon([
            (7, 7),
            (6, 7),
            (6, 6),
            (7, 6)
        ]),
        texture=Texture(
            id_=3,
            color='#ffd200',
            name='main_object'
        )
    )
)

### Define callback for plotting

In [25]:
class PlottingCallback(BaseCallback):
    """
    A callback class designed for plotting and visualizing various aspects of a simulation environment and agent behavior.

    This callback integrates with matplotlib to create a multi-panel figure that visualizes the environment, the agent's
    field of view (FOV), ego-centric representation, parietal window (PW) representation, and the agent's trajectory. It
    allows for interactive target setting through mouse clicks on the plot.

    Attributes:
        x_bvc (np.ndarray): The x-coordinates for plotting the BVC (Boundary Vector Cell) activity in the PW representation.
        y_bvc (np.ndarray): The y-coordinates for plotting the BVC activity in the PW representation.
        update_rate (int): The rate at which the plots are updated (every `update_rate` simulation steps).

    Args:
        x_bvc (np.ndarray): The x-coordinates for the BVC activity plot.
        y_bvc (np.ndarray): The y-coordinates for the BVC activity plot.
        update_rate (int, optional): The update rate for the plots. Defaults to 100.

    Methods:
        set_cache(cache: Any):
            Sets the cache for the callback and initializes required keys for plotting.

        on_click(event: KeyEvent):
            Handles mouse click events on the plot for setting movement and rotation targets.

        on_step_end(step: int):
            Updates the plots at the end of each simulation step, based on the specified update rate.

        on_simulation_end(n_cycles_passed: int):
            Closes the plot window at the end of the simulation.

        plot():
            Generates and updates all subplots within the figure.

        clean_axes():
            Clears and resets the axes for the next plot update.

        plot_environment():
            Plots the environment, including walls and objects.

        plot_fov():
            Plots the agent's field of view, showing visible walls and objects.

        plot_ego():
            Plots the ego-centric representation of walls and objects.

        plot_agent():
            Plots the agent's current position and direction.

        plot_pw():
            Plots the parietal window representation of walls and objects.

        plot_trajectory():
            Plots the agent's current trajectory towards a target.
    """
    def __init__(
        self,
        x_bvc: np.ndarray,
        y_bvc: np.ndarray,
        update_rate: int = 100
    ):
        """
        Initializes the PlottingCallback object.

        Args:
            x_bvc (np.ndarray): The x-coordinates for the BVC activity plot.
            y_bvc (np.ndarray): The y-coordinates for the BVC activity plot.
            update_rate (int, optional): The update rate for the plots. Defaults to 100.
        """
        super().__init__()
        self.x_bvc = x_bvc
        self.y_bvc = y_bvc
        self.update_rate = update_rate
        # Create a figure and a plot
        self.fig = plt.figure(figsize=(10, 5))
        # Create a GridSpec layout
        self.gs = GridSpec(12, 12, figure=self.fig)

        # Add the first subplot on the left side (spanning two rows)
        self.ax1 = self.fig.add_subplot(self.gs[:, :5])

        # Add the second subplot on the top right
        self.ax2 = self.fig.add_subplot(self.gs[:6, 6:])

        # Add the third subplot on the bottom right with 3D projection
        self.ax3 = self.fig.add_subplot(self.gs[6:, 6:9], projection='3d')

        # self.ax4 = self.fig.add_subplot(self.gs[7:10, 8:10])
        self.ax4 = self.fig.add_subplot(self.gs[6:, 9:12], projection='3d')

        # self.ax5 = self.fig.add_subplot(self.gs[10:, 6:12])

        # plt.subplots_adjust(bottom=0.2)

        # Set the limits of the plot
        self.ax1.set_xlim(-10, 10)
        self.ax1.set_ylim(-10, 10)
        self.ax2.set_xlim(-20, 20)
        self.ax2.set_ylim(-20, 20)
        self.ax3.set_axis_off()

        self.ax3.view_init(azim=-90, elev=90)
        self.ax3.set_axis_off()
        self.ax4.view_init(azim=-90, elev=90)
        self.ax4.set_axis_off()

        # # Connect the key press event to the handler
        # self.fig.canvas.mpl_connect('key_press_event', self.on_key)
        self.fig.canvas.mpl_connect('button_press_event', self.on_click)

    def set_cache(self, cache: Any):
        """
        Sets the cache for the callback and initializes required keys for plotting.

        Args:
            cache (Any): The cache to be set for the callback.
        """
        super().set_cache(cache)
        self.cache['move_target'] = None
        self.cache['rotate_target'] = None
        self.requires = [
            'env',
            'tc_gen',
            'move_target',
            'rotate_target',
            'walls_fov',
            'objects_fov',
            'walls_ego',
            'objects_ego',
            'walls_ego_segments',
            'objects_ego_segments',
            'walls_pw',
            'objects_pw',
            'position',
            'direction',
            'movement_schedule',
            'trajectory',
            'attend_to',
            'attention_priority'
        ]

    def on_click(self, event: MouseEvent):
        """
        Handles mouse click events on the plot for setting movement and rotation targets.

        Args:
            event (MouseEvent): The mouse click event on the plot.
        """
        self.plot()

        if event.inaxes is self.ax1:

            if event.button is MouseButton.LEFT:
                self.ax1.plot(event.xdata, event.ydata, 'rx')
                self.fig.canvas.draw()
                plt.pause(.00001)
                self.cache['move_target'] = event.xdata, event.ydata
                self.cache['rotate_target'] = None
            elif event.button is MouseButton.RIGHT:
                self.ax1.plot(event.xdata, event.ydata, 'co')
                self.fig.canvas.draw()
                plt.pause(.00001)
                self.cache['rotate_target'] = event.xdata, event.ydata
                self.cache['move_target'] = None

    def on_step_end(self, step: int):
        """
        Updates the plots at the end of each simulation step, based on the specified update rate.

        Args:
            step (int): The current simulation step.
        """

        if not step % self.update_rate:

            self.plot()

    def on_simulation_end(self, n_cycles_passed: int):
        """
        Closes the plot window at the end of the simulation.

        Args:
            n_cycles_passed (int): The number of simulation cycles that have passed.
        """
        plt.close()

    def plot(self):
        """
        Generates and updates all subplots within the figure.
        """
        self.clean_axes()
        self.plot_environment()
        if self.cache['move_target'] is not None:
            self.ax1.plot(*self.cache['move_target'], 'rx')
        if self.cache['rotate_target'] is not None:
            self.ax1.plot(*self.cache['rotate_target'], 'co')
        self.plot_trajectory()
        self.plot_fov()
        self.plot_ego()
        self.plot_agent()
        self.plot_pw()
        self.fig.canvas.draw()
        plt.pause(.00001)

    def clean_axes(self):
        """
        Clears and resets the axes for the next plot update.
        """
        self.ax1.clear(), self.ax2.clear(), self.ax3.clear(), self.ax4.clear()#, self.ax5.clear()
        self.ax1.set_xlim(-10, 10)
        self.ax1.set_ylim(-10, 10)
        self.ax2.set_xlim(-15, 15)
        self.ax2.set_ylim(-15, 15)

        self.ax3.view_init(azim=-90, elev=90)
        self.ax3.set_axis_off()
        self.ax4.view_init(azim=-90, elev=90)
        self.ax4.set_axis_off()

    def plot_environment(self):
        """
        Plots the environment, including walls and objects.
        """
        for obj in self.cache['env'].objects + self.cache['env'].walls:
            plot_polygon(obj.polygon, ax=self.ax1, alpha=0.5, linewidth=1)

    def plot_fov(self):
        """
        Plots the agent's field of view, showing visible walls and objects.
        """
        if self.cache['walls_fov']:
            for wall, poly in zip(self.cache['walls_fov'], self.cache['env'].walls):
                self.ax1.plot(wall[:, 0], wall[:, 1], 'o', color=poly.polygon.texture.color, markersize=2)
        if self.cache['objects_fov']:
            for i, (obj, poly) in enumerate(zip(self.cache['objects_fov'], self.cache['env'].objects)):
                if self.cache['attend_to'] is not None and i == self.cache['attend_to']:
                    self.ax1.plot(obj[:, 0], obj[:, 1], 'o', color='r', markersize=3)
                else:
                    self.ax1.plot(obj[:, 0], obj[:, 1], 'o', color=poly.polygon.texture.color, markersize=2)

    def plot_ego(self):
        """
        Plots the ego-centric representation of walls and objects.
        """
        _ = plot_arrow(np.pi/2, 0, -.75, ax=self.ax2)

        if self.cache['walls_ego_segments']:
            for segments, poly in zip(self.cache['walls_ego_segments'], self.cache['env'].walls):
                for seg in segments:
                    x_start, y_start, x_end, y_end = seg
                    self.ax2.plot([x_start, x_end], [y_start, y_end], color=poly.polygon.texture.color, linewidth=1)

        if self.cache['objects_ego_segments']:
            for segments, poly in zip(self.cache['objects_ego_segments'], self.cache['env'].objects):
                for seg in segments:
                    x_start, y_start, x_end, y_end = seg
                    self.ax2.plot([x_start, x_end], [y_start, y_end], color=poly.polygon.texture.color, linewidth=1)

    def plot_agent(self):
        """
        Plots the agent's current position and direction.
        """
        if self.cache['position'] is not None and self.cache['direction'] is not None:
            self.ax1.plot(*self.cache['position'], 'bo')
            self.ax1.arrow(*self.cache['position'], 0.5 * math.cos(self.cache['direction']), 0.5 * math.sin(self.cache['direction']))

    def plot_pw(self):
        """
        Plots the parietal window representation of walls and objects.
        """
        if self.cache['walls_pw'] is not None:
            self.ax3.plot_surface(
                self.x_bvc,
                self.y_bvc,
                np.reshape(self.cache['walls_pw'], (self.cache['tc_gen'].n_bvc_theta, self.cache['tc_gen'].n_bvc_r)),
                cmap='coolwarm'
            )

        if self.cache['objects_pw'] is not None:
            self.ax4.plot_surface(
                self.x_bvc,
                self.y_bvc,
                np.reshape(self.cache['objects_pw'], (self.cache['tc_gen'].n_bvc_theta, self.cache['tc_gen'].n_bvc_r)),
                cmap='coolwarm'
            )

    def plot_trajectory(self):
        """
        Plots the agent's current trajectory towards a target.
        """
        if self.cache['position'] is not None and \
            (not len(self.cache['trajectory']) or
            not (
                self.cache['move_target'] is not None
                and self.cache['move_target'] not in self.cache['trajectory']
            )):
            first_points = [self.cache['position'], self.cache['move_target']]\
                if self.cache['move_target'] not in self.cache['movement_schedule']\
                and self.cache['move_target'] is not None\
                else [self.cache['position']]
            all_points = first_points + self.cache['movement_schedule']
            if len(self.cache['movement_schedule']):
                self.ax1.plot(
                    self.cache['movement_schedule'][-1][0],
                    self.cache['movement_schedule'][-1][1],
                    'ro'
                )
            for from_, to in zip(all_points[:-1], all_points[1:]):
                self.ax1.plot(*zip(from_, to), 'g-')

### Compute plotting variables

In [26]:
fov_angle = np.pi
fov_manager = FOVManager(compiler.environment, fov_angle)
ego_manager = EgoManager(fov_manager)

cache = {'env': compiler.environment, 'tc_gen': tc_gen}
res = 0.01

polar_distance = calculate_polar_distance(tc_gen.r_max)
polar_angle = np.linspace(0, (tc_gen.n_bvc_theta + 1) * tc_gen.polar_ang_res, tc_gen.n_bvc_theta)
polar_distance, polar_angle = np.meshgrid(polar_distance, polar_angle)
pdist, pang = polar_distance, polar_angle
x_bvc, y_bvc = pol2cart(pdist, pang)
hd_polar_res = 2 * np.pi / n_hd
hd_angles = np.arange(0, 2 * np.pi+ hd_polar_res, hd_polar_res) + np.pi/2
hd_dist, hd_ang = np.meshgrid(np.array([1, 1.5]), hd_angles)
hd_x, hd_y = pol2cart(hd_dist, hd_ang)

### Running the cycle

In [28]:
%matplotlib qt

dynamics = DynamicsManager(
    res,
    callbacks=[
        MovementCallback(
            res,
            MovementManager(15, math.pi*4, (0, 0), 0)
        ),
        FOVCallback(fov_manager),
        EgoCallback(ego_manager),
        EgoSegmentationCallback(),
        ParietalWindowCallback(),
        PlottingCallback(x_bvc, y_bvc, 10),
        MovementSchedulerCallback(),
        TrajectoryCallback(
            AStarTrajectory(
                compiler.environment,
                n_points=5,
                method='quadratic',
                dx=.5,
                poly_increase_factor=1.2
            )
        ),
        AttentionCallback(RhythmicAttention(7, res, 5))
    ],
    cache=cache
)


for _ in dynamics(100):
    print('out: ', _)

switch
switch
switch
switch
switch
switch
switch
cycle
out:  (None, None, None, None, None, None, None, None, None)
switch
switch
switch
switch
switch
switch
switch
cycle
out:  (None, None, None, None, None, None, None, None, None)
switch
switch
switch
switch
switch
switch
switch
cycle
out:  (None, None, None, None, None, None, None, None, None)
switch
switch
switch
switch
switch
switch
switch
cycle
out:  (None, None, None, None, None, None, None, None, None)
switch
switch
switch
switch
switch
switch
switch
cycle
out:  (None, None, None, None, None, None, None, None, None)
switch
switch
switch
switch
switch
switch
switch
cycle
out:  (None, None, None, None, None, None, None, None, None)
switch
switch
switch
switch
switch
switch
switch
cycle
switch
out:  (None, None, None, None, None, None, None, None, None)
switch
switch
switch
switch
switch
switch
switch
cycle
out:  (None, None, None, None, None, None, None, None, None)
switch
switch
switch
switch
switch
switch
switch
cycle
out:  (Non

KeyboardInterrupt: 